In [1]:
import sys
from tqdm import tqdm

sys.path.append("../")
from calculus_path_mod.term_engine import *
from calculus_path_mod.reduction_strategy import *
from calculus_path_mod.terms import num_comparison, nat_numbers, arithm_ops, combinators, pairs, logic

from calculus_path_mod.terms.pseudonym import *

# Define Fibonacci term
An algorithm I took [here](https://whatthefunctional.wordpress.com/2019/02/21/a-brief-introduction-to-the-%CE%BB-calculus-part-2/)

In [3]:
def fib_term():
    # # FIB = (λf. (λp. (IF (ISZERO (FST p)) ZERO
    #              (IF (ISZERO (FST (FST p))) ONE
    #                  (PAIR (SND p)
    #                        (PLUS (FST p) (SND p))))))) (λf. (λp. (IF (ISZERO (FST p)) ZERO
    #                                                             (IF (ISZERO (FST (FST p))) ONE
    #                                                                 (PAIR (SND p)
    #                                                                 (PLUS (FST p) (SND p)))))))

    f, p = Var(), Var()
    f_, p_ = Atom(f), Atom(p)

    inner_term = multi_app_term(arithm_ops.plus_term(), App(pairs.first_term(), p_), App(pairs.second_term(), p_))
    inner_if_cond = App(num_comparison.iszero_term(), App(pairs.first_term(), App(pairs.first_term(), p_)))

    inner_term = multi_app_term(logic.ite_term(), inner_if_cond, nat_numbers.num_term(1), inner_term)
    inner_if_cond = App(num_comparison.iszero_term(), App(pairs.first_term(), p_))

    inner_term = multi_app_term(logic.ite_term(), inner_if_cond, nat_numbers.num_term(0), inner_term)
    return Lambda(f, Lambda(p, inner_term))

def fib_num_term():
    # FIB_NUM = (λn. (FST ( (FIB (PAIR n ZERO)) (PAIR ZERO ONE))))

    n = Var()
    n_ = Atom(n)
    fib_ = App(fib_term(), fib_term())
    inner_term = multi_app_term(pairs.pair_term(), n_, nat_numbers.num_term(0))
    inner_term = App(fib_, inner_term)
    inner_right_term = multi_app_term(pairs.pair_term(), nat_numbers.num_term(0), nat_numbers.num_term(1))
    inner_term = App(inner_term, inner_right_term)

    return Lambda(n, App(pairs.first_term(), inner_term))


In [4]:
strategy_lo = LOStrategy()
strategy_li = LIStrategy()
strategy_ri = RIStrategy()

def test_term(strategy_name, idx):
    strategy = None
    if strategy_name == "LO":
        strategy = strategy_lo
    elif strategy_name == "LI":
        strategy = strategy_li
    elif strategy_name == "RI":
        strategy = strategy_ri
    if not strategy:
        return
    fib_idx = App(fib_num_term(), nat_numbers.num_term(idx))
    norm_term, steps = fib_idx.normalize(strategy, is_limited=False)
    print(f"{strategy_name}: {steps} {norm_term.funky_str()}")

In [5]:
test_term("LO", 0)

LO: 20 (λx.(λy.x))


In [6]:
test_term("LO", 1)

LO: 42 (λx.(λy.y))


In [7]:
test_term("LO", 2)

LO: 39 (λx.(λy.x))


In [8]:
test_term("LO", 3)

LO: 40 (λx.x)


In [9]:
test_term("LO", 4)

LO: 41 (λx.(λy.y))


### With LI strategy

In [12]:
test_term("LI", 0)

LI: 63 (λx.(λy.x))


In [13]:
test_term("LI", 1)

LI: 67 (λx.(λy.y))


In [14]:
test_term("LI", 2)

LI: 67 (λx.(λy.x))


In [15]:
test_term("LI", 3)

LI: 69 (λx.x)


In [16]:
test_term("LI", 4)

LI: 71 (λx.(λy.y))
